# 加载数据

In [1]:
data_path='/home/xhsun/Desktop/KG/nlpcc2018/nlpcc2018.kbqg.train'
with open(data_path) as f:
    lines=f.readlines()

data=[[]]
for line in lines:
    if len(line.strip().split())<=1:
        data.append([])
    else:
        data[-1].append(line.strip())
if data[-1]==[]:
    del data[-1]
for example in data:
    assert len(example)==2
print(len(data))

24479


In [2]:
data[0]

['<triple id=1>\t机械设计基础 ||| 作者 ||| 杨可桢，程光蕴，李仲生',
 '<question id=1>\t《机械设计基础》这本书的作者是谁？']

In [3]:
all_examples=[]
for example in data:
    triplet,question=example
    triplet=triplet.split('\t')
    question=question.split('\t')
    assert len(question)==2 and question[0].startswith('<') and question[0].endswith('>') and len(triplet)==2
    question=question[1]
    triplet=triplet[1].split('|||')
    assert len(triplet)==3
    entity=triplet[0].strip()
    rel=triplet[1].strip()
    ans=triplet[2].strip()
    assert type(question)==str
    question=question.strip()
    if ('《' in question and '》' in question) and ('《' not in entity and '》' not in entity):
        #print(question,entity)
        entity='《'+entity+'》'
            
    if entity in question:
        start_id=question.find(entity)
        end_id=start_id+len(entity)
        question=question[:start_id]+'<'+entity+'>'+question[end_id:]
        all_examples.append({"question":question,'topic_entity':entity,'answer':ans,'relation':rel})

In [4]:
all_examples[0]

{'question': '<《机械设计基础》>这本书的作者是谁？',
 'topic_entity': '《机械设计基础》',
 'answer': '杨可桢，程光蕴，李仲生',
 'relation': '作者'}

# 构造实体识别数据

In [5]:
labels=[]
sentences=[]
examples=[]
found_num=0
for example in data:
    triplet,question=example
    triplet=triplet.split('\t')
    question=question.split('\t')
    assert len(question)==2 and question[0].startswith('<') and question[0].endswith('>') and len(triplet)==2
    question=question[1]
    triplet=triplet[1].split('|||')
    assert len(triplet)==3
    entity=triplet[0]
    assert type(question)==str
    question=question.strip()
    found=False
    positions=[]
    
    entity=entity.strip()
    if entity in question:
        start_id=question.find(entity)
        end_id=start_id+len(entity)
        positions.append((start_id,end_id))
        found=True
        
    if found:
        found_num+=1
        sentences.append(question)
        label=['O']*len(question)
        for each_span in positions:
            start_id,end_id=each_span
            for i in range(start_id,end_id):
                if i==start_id:
                    label[i]='B-ent'
                else:
                    label[i]='I-ent'
        labels.append(label)
        examples.append(example)

In [6]:
print(len(sentences),len(labels))
for i in range(10):
    print(sentences[i],labels[i],examples[i])
    print('-'*100)

24000 24000
《机械设计基础》这本书的作者是谁？ ['O', 'B-ent', 'I-ent', 'I-ent', 'I-ent', 'I-ent', 'I-ent', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'] ['<triple id=1>\t机械设计基础 ||| 作者 ||| 杨可桢，程光蕴，李仲生', '<question id=1>\t《机械设计基础》这本书的作者是谁？']
----------------------------------------------------------------------------------------------------
《高等数学》是哪个出版社出版的？ ['O', 'B-ent', 'I-ent', 'I-ent', 'I-ent', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'] ['<triple id=2>\t高等数学 ||| 出版社 ||| 武汉大学出版社', '<question id=2>\t《高等数学》是哪个出版社出版的？']
----------------------------------------------------------------------------------------------------
《线性代数》这本书的出版时间是什么？ ['O', 'B-ent', 'I-ent', 'I-ent', 'I-ent', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'] ['<triple id=3>\t线性代数 ||| 出版时间 ||| 2013-12-30', '<question id=3>\t《线性代数》这本书的出版时间是什么？']
----------------------------------------------------------------------------------------------------
安德烈是哪个国家的人呢？ ['B-ent', 'I-ent', 'I-ent', 'O', 'O', 'O', 'O', '

# 导入包

In [7]:
import torch,json
import sys,os
import pandas as pd
import random
import numpy as np
from nlp_basictasks.tasks import Ner
from nlp_basictasks.evaluation import nerEvaluator
from nlp_basictasks.readers.ner import InputExample
from collections import Counter
from tqdm import tqdm
from copy import deepcopy
import pandas
import re

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


# 构造训练集和测试集

In [11]:
index=list(range(len(sentences)))
random.shuffle(index)
test_nums=int(len(index)*0.2)
print(test_nums)
train_sentences=[]
train_labels=[]
train_raws=[]
for i in range(len(index)-test_nums):
    train_sentences.append(list(sentences[index[i]]))
    train_labels.append(labels[index[i]])
    train_raws.append(examples[index[i]])
    
k=10
print(train_sentences[k],train_labels[k],train_raws[k])

test_sentences=[]
test_labels=[]
test_raws=[]
for i in range(len(index)-test_nums,len(index)):
    test_sentences.append(list(sentences[index[i]]))
    test_labels.append(labels[index[i]])
    test_raws.append(examples[index[i]])
    
print(test_sentences[k],test_labels[k],test_raws[k])

4800
['你', '知', '道', '普', '惠', 'r', '-', '1', '3', '4', '0', '发', '动', '机', '是', '什', '么', '型', '式', '吗', '？'] ['O', 'O', 'O', 'B-ent', 'I-ent', 'I-ent', 'I-ent', 'I-ent', 'I-ent', 'I-ent', 'I-ent', 'I-ent', 'I-ent', 'I-ent', 'O', 'O', 'O', 'O', 'O', 'O', 'O'] ['<triple id=4355>\t普惠r-1340发动机 ||| 型式 ||| 活塞发动机', '<question id=4355>\t你知道普惠r-1340发动机是什么型式吗？']
['你', '知', '道', '星', '尘', '龙', '的', '防', '御', '力', '是', '多', '少', '吗', '？'] ['O', 'O', 'O', 'B-ent', 'I-ent', 'I-ent', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'] ['<triple id=23551>\t星尘龙 ||| 防御 ||| 2000', '<question id=23551>\t你知道星尘龙的防御力是多少吗？']


In [12]:
train_examples=[]
for seq_in,seq_out in zip(train_sentences,train_labels):
    train_examples.append(InputExample(seq_in=seq_in,seq_out=seq_out))

test_examples=[]
for seq_in,seq_out in zip(test_sentences,test_labels):
    test_examples.append(InputExample(seq_in=seq_in,seq_out=seq_out))

In [13]:
print(len(train_examples),len(test_examples))

19200 4800


In [14]:
label2id={'[PAD]':0,'O':1,'B-ent':2,'I-ent':3}

# 定义模型加载路径

In [17]:
model_path='/home/xhsun/Desktop/huggingfaceModels/chinese-roberta-wwm/'
output_path='/home/xhsun/Desktop/tmpFiles/NER'
ner_model=Ner(model_path,label2id=label2id,use_crf=True,use_bilstm=False,device='cuda',batch_first=True,
              tensorboard_logdir='/home/xhsun/Desktop/tmpFiles/NER/log.txt')

2021-12-10 15:35:47 - INFO - __init__ - 53 : The label2id is
 {"[PAD]": 0, "O": 1, "B-ent": 2, "I-ent": 3}
2021-12-10 15:35:47 - INFO - __init__ - 270 : Loading model from /home/xhsun/Desktop/huggingfaceModels/chinese-roberta-wwm/, which is from huggingface model
2021-12-10 15:35:47 - INFO - get_config_dict - 177 : loading configuration file /home/xhsun/Desktop/huggingfaceModels/chinese-roberta-wwm/config.json
2021-12-10 15:35:47 - INFO - from_pretrained - 404 : loading bert model file /home/xhsun/Desktop/huggingfaceModels/chinese-roberta-wwm/
2021-12-10 15:35:47 - INFO - from_pretrained - 423 : BertConfig has been loaded from /home/xhsun/Desktop/huggingfaceModels/chinese-roberta-wwm/config.json
2021-12-10 15:35:48 - INFO - from_pretrained - 125 : loading vocabulary file /home/xhsun/Desktop/huggingfaceModels/chinese-roberta-wwm/vocab.txt
2021-12-10 15:35:48 - INFO - __init__ - 70 : Use pytorch device: cuda
2021-12-10 15:35:48 - INFO - __init__ - 72 : Using BiLSTM ? : False
2021-12-10 1

In [18]:
from torch.utils.data import DataLoader
batch_size=64
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)

evaluator=nerEvaluator(label2id=label2id,seq_in=test_sentences,seq_out=test_labels,write_csv=True)

2021-12-10 15:35:59 - INFO - __init__ - 39 : Total evaluate nums : 4800
2021-12-10 15:35:59 - INFO - __init__ - 40 : input is string : False, input is list : True
2021-12-10 15:35:59 - INFO - __init__ - 41 : seq in and out like : 
['请', '问', '阿', '明', '托', '雷', '·', '范', '范', '尼', '的', '总', '统', '是', '谁', '？']	['O', 'O', 'B-ent', 'I-ent', 'I-ent', 'I-ent', 'I-ent', 'I-ent', 'I-ent', 'I-ent', 'O', 'O', 'O', 'O', 'O', 'O']
2021-12-10 15:35:59 - INFO - __init__ - 42 : In this evaluator, slot contains ([PAD] O B-ent I-ent)


# 模型训练

In [20]:
ner_model.fit(train_dataloader=train_dataloader,evaluator=evaluator,epochs=5,output_path=output_path)

2021-12-10 15:36:23 - INFO - fit - 155 : label2id has been saved in /home/xhsun/Desktop/tmpFiles/NER/label2id.json
2021-12-10 15:36:23 - INFO - fit - 164 : 一个epoch 下，每隔60个step会输出一次loss，每隔150个step会评估一次模型


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

0it [00:00, ?it/s]

2021-12-10 15:36:23 - INFO - __call__ - 59 : nerEvaluator: Evaluating the model on  dataset in epoch 0 after 1 steps:


  0%|          | 0/150 [00:00<?, ?it/s]

...

2021-12-10 15:39:23 - INFO - save_pretrained - 509 : Model weights saved in /home/xhsun/Desktop/tmpFiles/NER/BERT/pytorch_model.bin
2021-12-10 15:39:23 - INFO - save_pretrained - 150 : Configuration saved in /home/xhsun/Desktop/tmpFiles/NER/BERT/config.json
2021-12-10 15:39:23 - INFO - save_vocab - 51 : Vocab saved in /home/xhsun/Desktop/tmpFiles/NER/BERT/vocab.txt
2021-12-10 15:39:23 - INFO - fit - 239 : In epoch 4, training_step 0, the eval score is 0.9797065251326881, previous eval score is 0.9774264017476334, model has been saved in /home/xhsun/Desktop/tmpFiles/NER
2021-12-10 15:39:30 - INFO - fit - 217 : Epoch : 4, train_step : 300/1500, loss_value : 0.0020884841035391827 
2021-12-10 15:39:37 - INFO - fit - 217 : Epoch : 4, train_step : 600/1500, loss_value : 0.0016744306912490478 
2021-12-10 15:39:41 - INFO - __call__ - 59 : nerEvaluator: Evaluating the model on  dataset in epoch 4 after 151 steps:


  0%|          | 0/150 [00:00<?, ?it/s]

2021-12-10 15:39:44 - INFO - fit - 243 : No improvement over previous best eval score (0.978458 vs 0.979707), patience = 7


              precision    recall  f1-score   support

         ent     0.9775    0.9794    0.9785      4800

   micro avg     0.9775    0.9794    0.9785      4800
   macro avg     0.9775    0.9794    0.9785      4800
weighted avg     0.9775    0.9794    0.9785      4800



2021-12-10 15:39:48 - INFO - fit - 217 : Epoch : 4, train_step : 900/1500, loss_value : 0.0030521146824564007 
2021-12-10 15:39:55 - INFO - fit - 217 : Epoch : 4, train_step : 1200/1500, loss_value : 0.0016418134572935136 
